<a href="https://colab.research.google.com/github/BryanCoboF/optimizacionCosecha/blob/main/sitio/optCosechaSitiov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuración Inicial
Llamada a librerias

In [3]:
#Configuracion de librerias
%config IPCompleter.greedy=True
!pip install pulp
import pandas as pd
import numpy as np
import xlrd
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import pulp
from pulp import LpVariable, LpProblem, LpMaximize, LpMinimize
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive



Carga de datos de Excel desde drive

In [38]:
from google.colab import files
from google.colab import drive

drive.mount('/content/drive') # Permite ingresar a drive desde las carpetas de drive

df = pd.read_excel("/content/drive/MyDrive/PB/BaseOptimizacion.xlsx") # Carga el excel de datos de proyeccion, si se necesita cambiar el archivo se debe modificar el link



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ingreso de datos deseados


1.   Volumen deseado por cada mes
3.   Densidad deseada por cada mes
2.   Edad minima y maxima deseada


In [93]:
# Volumen deseado cada mes
vecVolDeseado = []
vecVolR11minDeseado = []
vecVolR11maxDeseado = []
vecVolmaxDeseado = []
vecDensminDeseada = []
vecDensmaxDeseada = []

semanasMes = [4,4,5,4,5,4,4,5,4,4,5,4] #Numero de semanas por cada mes

#Ingreso de volumen anual deseado
volDes = float(input("Ingrese el volumen deseado 2024: "))

# Calculo del volumen deseado para cada mes en base al deseado anual y las semanas que tiene cada mes.
for i in range(12):
  vecVolDeseado.append((volDes/52)*semanasMes[i])

print(vecVolDeseado)
print(sum(vecVolDeseado))

#Ingreso de edad deseada
edadMinDes = float(input("Ingrese la edad minima de cosecha : "))
areaMaxCosecha = float(input("Ingrese el area máxima cosechable por sitio cada mes: "))

Ingrese el volumen deseado 2024: 27000000
[2076923.076923077, 2076923.076923077, 2596153.846153846, 2076923.076923077, 2596153.846153846, 2076923.076923077, 2076923.076923077, 2596153.846153846, 2076923.076923077, 2076923.076923077, 2596153.846153846, 2076923.076923077]
26999999.999999996
Ingrese la edad minima de cosecha : 26
Ingrese el area máxima cosechable por sitio cada mes: 100


OBTENCION  DE DATOS DEL EXCEL
Creación de Diccionarios Y VECTORES


In [94]:
# Obtencion de datos de sitio-año de plantacion
colSitioAnio = df['Sitio-AñoPlantación']

# Obtencion de datos area neta
colAreaNeta = df['Sum of Área Neta del Lote (Ha)']

# ------------- Obtencion de datos de edad -----------------
# Recibe data frame con los datos de cada mes del excel
# Crea un diccionario por cada mes con los datos por sitio-lote
#Enero
colEdadEne = df['Sum of Edad pond ENE']
#Febrero
colEdadFeb = df['Sum of Edad pond FEB']
#Marzo
colEdadMar = df['Sum of Edad pond MAR']
#Abril
colEdadAbr = df['Sum of Edad pond ABR']
#Mayo
colEdadMay = df['Sum of Edad pond MAY']
#Junio
colEdadJun = df['Sum of Edad pond JUN']
#Julio
colEdadJul = df['Sum of Edad pond JUL']
#Agosto
colEdadAgo = df['Sum of Edad pond AGO']
#Septiembre
colEdadSep = df['Sum of Edad pond SEP']
#Octubre
colEdadOct = df['Sum of Edad pond OCT']
#Noviembre
colEdadNov = df['Sum of Edad pond NOV']
#Diciembre
colEdadDic = df['Sum of Edad pond DIC']


#--------OBTENCION DE DATOS DE RENDIMIENTO ----------------
# Obtiene los datos de rendimiento del data frame de excel de cada mes proyectado
#Enero
colRendProyEne = df['Sum of Rend pond ENE']
#Febrero
colRendProyFeb = df['Sum of Rend pond FEB']
#Marzo
colRendProyMar = df['Sum of Rend pond MAR']
#Abril
colRendProyAbr = df['Sum of Rend pond ABR']
#Mayo
colRendProyMay = df['Sum of Rend pond MAY']
#Junio
colRendProyJun = df['Sum of Rend pond JUN']
#Julio
colRendProyJul = df['Sum of Rend pond JUL']
#Agosto
colRendProyAgo = df['Sum of Rend pond AGO']
#Septiembre
colRendProySep = df['Sum of Rend pond SEP']
#Octubre
colRendProyOct = df['Sum of Rend pond OCT']
#Noviembre
colRendProyNov = df['Sum of Rend pond NOV']
#Dicimebre
colRendProyDic = df['Sum of Rend pond DIC']

#------- OBTENCION DE DATOS DE ACCESIBILIDAD -------------------
#Enero
colAccesoEne = df['Sum of AccesibilidadENE']
#Febrero
colAccesoFeb = df['Sum of AccesibilidadFEB']
#Marzo
colAccesoMar = df['Sum of AccesibilidadMAR']
#Abril
colAccesoAbr = df['Sum of AccesibilidadABR']
#Mayo
colAccesoMay = df['Sum of AccesibilidadMAY']
#Junio
colAccesoJun = df['Sum of AccesibilidadJUN']
#Julio
colAccesoJul = df['Sum of AccesibilidadJUL']
#Agosto
colAccesoAgo = df['Sum of AccesibilidadAGO']
#Septiembre
colAccesoSep = df['Sum of AccesibilidadSEP']
#Octubre
colAccesoOct = df['Sum of AccesibilidadOCT']
#Novimebre
colAccesoNov = df['Sum of AccesibilidadNOV']
#Diciembre
colAccesoDic = df['Sum of AccesibilidadDIC']



# ---------CREACION DE MATRICES POR CADA MES DEL AÑO ------------
filas = len(colSitioAnio)  # Número de filas
columnas = 12  # Número de columnas (meses del siguiente año)

#//MATRICES//
# Inicializa las matrices n filas y 12 columnas
#Todos los valores se inicializan  con cero  /// Todas las matrices son del mismo tamaño
matrizEdad = [[0] * columnas for _ in range(filas)]
matrizRend = [[0] * columnas for _ in range(filas)]
matrizRendAux = [[0] * columnas for _ in range(filas)]
matrizAcceso = [[0] * columnas for _ in range(filas)]

# -------------Asignacion de valores a cada mes de las matrices---------
# Matriz de Edades por cada mes del 2024
# Matriz  de rendimientos
# Matriz de Accesibilidad

for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    if i == 0:
      matrizEdad[j][i] = colEdadEne[j]
      matrizRend[j][i] = colRendProyEne[j]
      matrizAcceso[j][i] = colAccesoEne [j]

    if i == 1:
      matrizEdad[j][i] = colEdadFeb[j]
      matrizRend[j][i] = colRendProyFeb[j]
      matrizAcceso[j][i] = colAccesoFeb [j]

    if i == 2:
      matrizEdad[j][i] = colEdadMar[j]
      matrizRend[j][i] = colRendProyMar[j]
      matrizAcceso[j][i] = colAccesoMar [j]

    if i == 3:
      matrizEdad[j][i] = colEdadAbr[j]
      matrizRend[j][i] = colRendProyAbr[j]
      matrizAcceso[j][i] = colAccesoAbr [j]

    if i == 4:
      matrizEdad[j][i] = colEdadMay[j]
      matrizRend[j][i] = colRendProyMay[j]
      matrizAcceso[j][i] = colAccesoMay [j]

    if i == 5:
      matrizEdad[j][i] = colEdadJun[j]
      matrizRend[j][i] = colRendProyJun[j]
      matrizAcceso[j][i] = colAccesoJun [j]

    if i == 6:
      matrizEdad[j][i] = colEdadJul[j]
      matrizRend[j][i] = colRendProyJul[j]
      matrizAcceso[j][i] = colAccesoJul [j]

    if i == 7:
      matrizEdad[j][i] = colEdadAgo[j]
      matrizRend[j][i] = colRendProyAgo[j]
      matrizAcceso[j][i] = colAccesoAgo [j]

    if i == 8:
      matrizEdad[j][i] = colEdadSep[j]
      matrizRend[j][i] = colRendProySep[j]
      matrizAcceso[j][i] = colAccesoSep [j]

    if i == 9:
      matrizEdad[j][i] = colEdadOct[j]
      matrizRend[j][i] = colRendProyOct[j]
      matrizAcceso[j][i] = colAccesoOct [j]

    if i == 10:
      matrizEdad[j][i] = colEdadNov[j]
      matrizRend[j][i] = colRendProyNov[j]
      matrizAcceso[j][i] = colAccesoNov [j]

    if i == 11:
      matrizEdad[j][i] = colEdadDic[j]
      matrizRend[j][i] = colRendProyDic[j]
      matrizAcceso[j][i] = colAccesoDic [j]



Creacion del diccionario con todas las edades

RESTRICCION DE EDAD DE COSECHA Y ACCESIBILIDAD

In [95]:
# Si el sitio cumple con los requerimientos de edad y accesibilidad en ese mes se asgina el rendimiento proyectado,
# caso contrario se asigna 0
# Pregunta primero si desea aplicar las restricciones

restEdad = float(input("Desea restringir edad? (SI: Ingrese 1,  NO: Ingrese 0) : "))
restAcceso = float(input("Desea restringir accesibilidad? (SI: Ingrese 1,  NO: Ingrese 0) : "))


if restEdad == 1 and restAcceso == 1:
  # Restricciones completas --- Se corre solo si se ingreso que se desea restringir edad y acceso
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizAcceso[j][i] == 1 and matrizEdad[j][i] >= 26:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 0 and restAcceso == 1:
  # Restriccion de acceso unicamente
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizAcceso[j][i] == 1:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 1 and restAcceso == 0:
  # Restriccion solo de edad
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizEdad[j][i] >= 26:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 0 and restAcceso == 0:
  # Sin restricciones
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      matrizRendAux[j][i] = matrizRend[j][i]

Desea restringir edad? (SI: Ingrese 1,  NO: Ingrese 0) : 0
Desea restringir accesibilidad? (SI: Ingrese 1,  NO: Ingrese 0) : 0


-------------------------------------------

MODELO OPTIMIZACION

------------------------------------------------------------

In [ ]:
#****************************** MODELO*********************************************************************************
#-------------Definicion del modelo-----------------
optCosecha = pulp.LpProblem('OptimizacionCosecha',pulp.LpMinimize) #Inicialmente busca minimizar el area de cosecha para obtener el volumen necesario para Enero


# ------------VARIABLE DE DECISION (AREA A COSECHAR)-------------------
area_a_cosechar = [[pulp.LpVariable(f'x{i}_{j}', lowBound=0, cat='Continuous') for j in range(columnas)] for i in range(filas)]


#--------------FUNCION OBJETIVO-------------------------
#optCosecha += sum([rend_EneAux[i]*area_a_cosechar_Ene[i]+rend_FebAux[i]*area_a_cosechar_Feb[i]+ rend_MarAux[i]*area_a_cosechar_Mar[i]+rend_AbrAux[i]*area_a_cosechar_Abr[i]+rend_MayAux[i]*area_a_cosechar_May[i]+rend_JunAux[i]*area_a_cosechar_Jun[i]+rend_JulAux[i]*area_a_cosechar_Jul[i]+rend_AgoAux[i]*area_a_cosechar_Ago[i]+rend_SepAux[i]*area_a_cosechar_Sep[i]+rend_OctAux[i]*area_a_cosechar_Oct[i]+rend_NovAux[i]*area_a_cosechar_Nov[i]+rend_DicAux[i]*area_a_cosechar_Dic[i] for i in colSitioAnio])
optCosecha += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar[i][j] for j in range(columnas) for i in range(filas))


#--------------RESTRICCIONES----------------------------
# 1) -- Restringe las areas a cosechar de cada sitio y mes para que no sean mayores al area neta
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    optCosecha += area_a_cosechar[j][i] >= 0
    optCosecha += area_a_cosechar[j][i] <= colAreaNeta[j]
    optCosecha += area_a_cosechar[j][i] <= areaMaxCosecha

# 2) -- Restringe que la suma del area a cosecha de cada sitio en todo el año sea menor o igual al area neta
for j in range(filas): # recorre filas
  optCosecha += sum(area_a_cosechar[j]) <= colAreaNeta[j]

# 3) Restriccion de volumen anual
optCosecha += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar[i][j] for j in range(columnas) for i in range(filas)) <= volDes

# 4) Restringe los volumenes de cada mes dependiendo del volumen deseado
for j in range(columnas):
  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) <= vecVolDeseado[j]
  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) >= vecVolDeseado[j]-50000
#----------------------------------------------------------

#Resolucion
optCosecha.solve()
#****************************** MODELO*********************************************************************************


#--------------Impresion de resultados preliminares--------------------------------------------------------------------
a=1
for j in range(filas): # recorre filas
  # Imprime cada sitio con el area asignada en el mes que se va a cosechar
  print(a, ") Areas ENE ", colSitioAnio[j], "-> " ,round(area_a_cosechar[j][0].value(),1), ", " ,round(area_a_cosechar[j][1].value(),1), ", " ,round(area_a_cosechar[j][2].value(),1),", " ,round(area_a_cosechar[j][3].value(),1),", " ,round(area_a_cosechar[j][4].value(),1),", " ,round(area_a_cosechar[j][5].value(),1),", " ,round(area_a_cosechar[j][6].value(),1),", " ,round(area_a_cosechar[j][7].value(),1), ", " ,round(area_a_cosechar[j][8].value(),1),", " ,round(area_a_cosechar[j][9].value(),1),", " ,round(area_a_cosechar[j][10].value(),1),", " ,round(area_a_cosechar[j][11].value(),1))
  a = a+1

for b in range(columnas):
  #Imprime el volumen a entregar cada mes
  print("Vol> ", b+1, "->", sum(matrizRend[i][b]*area_a_cosechar[i][b].value() for i in range(filas) ))

#Imprime el volumen total anual
print(sum(matrizRend[i][j]*area_a_cosechar[i][j].value() for j in range(columnas) for i in range(filas) ))
#-----------------------------------------------------------------------------------------------------------------------



Correccion de errores

In [ ]:
# Matriz que obtiene los resultados de las areas optimizadas de cosecha
matrizAreaRes = [[0] * columnas for _ in range(filas)]
# Impresion de errores
countErrores = 0
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    matrizAreaRes [j][i] = area_a_cosechar[j][i].value()
    if area_a_cosechar[j][i].value() <0:
      countErrores = countErrores + 1
print("Errores-> ", countErrores)

# Remanentes
for j in range(filas): # recorre filas
  if round(sum(matrizAreaRes[j]),2)<= round(colAreaNeta[j],2)-1:
    print(colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2), "-- Remanente 2025 -- ",round(colAreaNeta[j],2)-round(sum(matrizAreaRes[j]),2))
  else:
    print(colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2))

Creacion del Excel con resultados


In [ ]:
columnasRes = 13
matrizResultados = [[0] * columnasRes for _ in range(filas)]

#Asignación de nombres de sitio a la matriz de resultados
for h in range(filas):
  matrizResultados[h][0] = colSitioAnio[h]
k=1

# Asignación de resultados a la matriz
for k in range(columnasRes):
  for h in range(filas):
    if k>0:
      matrizResultados[h][k] = area_a_cosechar[h][k-1].value()



for f in range(filas):
  print(matrizResultados[f])

#Nombres de las columnas del excel
nomColRes = ['Sitio-Año de plantación', 'Área ENE 2024', 'Área FEB 2024','Área MAR 2024', 'Área ABR 2024','Área MAY 2024','Área JUN 2024','Área JUL 2024','Área AGO 2024', 'Área SEP 2024','Área OCT 2024','Área NOV 2024','Área DIC 2024']

#Crea el data frame con los nombres de columnas y la matriz de resultados
df1 = pd.DataFrame(matrizResultados, columns=nomColRes)

# Path donde se crea el archivo resOptimizacion
pathExcel = '/content/drive/MyDrive/PB/resOptimizacion.xlsx'

#Guarda el data frame en el excel
df1.to_excel(pathExcel, index=False)

